In [5]:
import numpy as np
import matplotlib.pyplot as plt
from database_methods import Table, to_show, head, process_data
import sqlite3
con = sqlite3.connect('database_results.db')
cursor = con.cursor()

In [6]:
mas = Table(connection=con, name='masters')
exp = Table(con, name = 'experiments')
meas = Table(con, name = 'measurements')

In [ ]:
print(mas.columns)
print(exp.columns)
print(meas.columns)
print('-'*80)
mas.show_all()
exp.show_all()

In [17]:
con.commit()

In [18]:
# 
columns = 'MASTER,  AVG(DIP), AVG(INP), AVG(INP/IBGPX), AVG(CON), AVG(INPPX), AVG(IBGPX)'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
GROUP BY MASTER
ORDER BY DIP DESC
;
   """.format(columns))

head(columns)

for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])


MASTER	 AVG(DI	AVG(INP	AVG(INP	AVG(CON	AVG(INP	AVG(IBG
L3	740.00	18.53	54.94	3.31	1.15	0.3505
Q4	727.80	24.76	68.44	4.60	1.67	0.3943
K5	658.00	14.79	48.99	5.44	1.64	0.3078
K4	647.00	6.18	24.67	3.32	0.9056	0.2691
P3	615.00	5.25	20.50	7.27	1.81	0.2621
M5	606.00	2.54	11.68	6.60	1.42	0.2196


In [35]:
# 
columns = 'MEAS.ID, MASTER, DATE, FOLDER, FILE, WAVELENGTH, NP, DIP, COUNT, RATE, CON, COUNT, X*Y, INP, INPPX, IBGPX, STD, EXP_TYPE,  NOTES'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
WHERE MASTER = 'Q4' AND DATE='20-04-20' AND WAVELENGTH = 732 AND NP = 80
ORDER BY CON DESC
;
   """.format(columns))

head(columns)

i=0
for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])
    i+=1
    
    
print('---\n{}'.format(i))

MEAS.ID	MASTER	DATE	FOLDER	FILE	WAVELEN	NP	DIP	COUNT	RATE	CON	COUNT	X*Y	INP	INPPX	IBGPX	STD	EXP_TYP	 NOTES
76	Q4	20-04-2	20_04_2	raw_02_	732	80	729	138	47.93	11.33	138	225.36	25.61	3.24	0.2904	0.4365	-	-
85	Q4	20-04-2	20_04_2	raw_17_	732	80	729	226	61.15	6.50	226	306.35	25.23	1.76	0.2741	0.4109	strep	-
82	Q4	20-04-2	20_04_2	raw_15_	732	80	729	101	31.38	4.65	101	469.57	29.46	1.27	0.2729	0.4068	sh	-
---
3


In [33]:
meas.commit()

In [25]:
folder = '20_04_20_Q4/'
file = 'raw_17_1'

columns = 'COUNT(MEAS.ID), MEAS.ID'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID 
WHERE FOLDER = '{}' AND FILE = '{}'
;
   """.format(columns, folder, file)
                  )

In [26]:
[d for d in data][0]

(3, 79)

In [20]:
b

In [28]:
columns = 'NP, WAVELENGTH, AR_TH, AR_NOISE, EXC_THS, AR_MIN, AR_COND, exp_type'
data = con.execute("""
SELECT {} FROM 'measurements'
WHERE WAVELENGTH=732
ORDER BY WAVELENGTH ASC;
   """.format(columns))

print(columns.replace(', ','\t'))
for row in data:
    print(str(row).replace(', ', '\t').replace("'", "")[1:-1])

NP	WAVELENGTH	AR_TH	AR_NOISE	EXC_THS	AR_MIN	AR_COND	exp_type
80	732	60.0	0.001	[2]	10	2	-
60	732	59.0	0.001	[2]	10	2	-
60	732	46.0	0.001	[2]	10	2	sh
40	732	12.0	0.001	[1.1]	2	2	-
80	732	100.0	0.001	[2]	2	2	-
60	732	20.0	0.001	[2]	2	2	-
80	732	40.0	0.001	[2]	2	2	strep
147	732	8.0	0.001	[2]	2	2	ps
40	732	9.0	0.001	[2]	6	2	-
80	732	31.0	0.001	[2]	6	2	sh


In [8]:
con.execute("""
UPDATE 'measurements' SET NOTES = 'weak' WHERE COUNT = 165
""")

In [215]:
meas.commit()

In [51]:
con.close()